In [23]:
# force geopandas to use shapely instead of pygeos
# import os
import pygeos
# os.environ['USE_PYGEOS'] = '0'

from shapely import wkt
# from shapely.geometry import Point
import geopandas as gpd
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt


# Ingest Data

In [24]:
df = pd.read_csv("../data/data_2022_with_geom.csv")

In [25]:
df['geom'] = df.geom.apply(wkt.loads)
df = gpd.GeoDataFrame(df, geometry='geom')
df.crs = 'EPSG:4326'

In [26]:
df['Citation Issued DateTime'] = pd.to_datetime(df['Citation Issued DateTime'])

In [27]:
df.head()

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom
0,950226616,2022-04-20 12:10:00,TRC7.2.22,STR CLEAN,1318 FILBERT ST,CA,V507699,84.0,05/01/2022 12:00:00 AM,POINT (-122.42120 37.80002)
1,952126114,2022-06-17 16:12:00,TRC7.2.20,RES/OT,1318 FILBERT ST,CA,BV44D97,96.0,07/17/2022 12:00:00 AM,POINT (-122.42120 37.80002)
2,950441903,2022-04-23 15:13:00,V5200,NO PLATES,1318 FILBERT ST,CA,V507699,121.0,05/18/2022 12:00:00 AM,POINT (-122.42120 37.80002)
3,948460144,2022-02-22 11:54:00,TRC7.2.20,RES/OT,1318 FILBERT ST,TX,HVT6737,96.0,05/23/2022 12:00:00 AM,POINT (-122.42120 37.80002)
4,948169644,2022-02-17 13:39:00,TRC7.2.20,RES/OT,1318 FILBERT ST,OH,HQA6554,96.0,04/03/2022 12:00:00 AM,POINT (-122.42120 37.80002)


In [28]:
df['citation_day'] = df['Citation Issued DateTime'].dt.day_name()

In [29]:
# sort by date

df.sort_values('Citation Issued DateTime', inplace=True)

# Street Sections

In [30]:
# load and process streets information

street_df = pd.read_csv('../data/street_sweeping.csv')
street_df = street_df[~street_df.Line.isnull()]
street_df.loc[:, 'Line'] = street_df.Line.apply(wkt.loads)
street_df = gpd.GeoDataFrame(street_df, geometry='Line')
street_df.crs = 'EPSG:4326'

In [31]:
street_df['segment_midpoint_x'] = street_df['Line'].apply(lambda line: line.centroid.x)
street_df['segment_midpoint_y'] = street_df['Line'].apply(lambda line: line.centroid.y)

In [32]:
# spatial join - may need to reproject to flat geometry but seems like points are so close together that it would make a neglible difference. 

df_joined = gpd.sjoin_nearest(df, street_df, distance_col='dist', how='left')

/Users/timtan/opt/anaconda3/lib/python3.9/site-packages/geopandas/array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [33]:
# drop duplicates from spatial join, still need to figure out why this happens

df_joined = df_joined.drop_duplicates(subset='Citation Number', keep="first")
assert df_joined.shape[0] == df_joined.shape[0]

In [34]:
df_joined.head()

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,...,Week1,Week2,Week3,Week4,Week5,Holidays,BlockSweepID,segment_midpoint_x,segment_midpoint_y,dist
171693,PD36185715,2022-01-01 02:45:00,PK6.01G,NOPRK 10P6,1000 GREAT HIGHWAY,CA,7GXP111,23.0,03/15/2022 12:00:00 AM,POINT (-122.51054 37.76982),...,1,1,1,1,1,1,1601828,-122.510437,37.767239,0.000233
1119962,PD36997575,2022-01-01 02:45:00,V5204A,REG TABS,2251 GREENWICH,CA,8XOD966,341.0,02/06/2023 12:00:00 AM,POINT (-122.43672 37.79858),...,1,0,1,0,0,0,1638225,-122.436596,37.798382,0.000214
394815,946687140,2022-01-01 07:00:00,V22500E,DRIVEWAY,665 EDDY ST,CA,7SCR930,110.0,02/05/2023 12:00:00 AM,POINT (-122.41862 37.78289),...,1,1,1,1,1,0,1630919,-122.418355,37.783206,0.000282
273898,PD36969144,2022-01-01 08:16:00,V5204A,REG TABS,100 YACHT ROAD,CA,7NVL746,121.0,11/20/2022 12:00:00 AM,POINT (-122.44857 37.80643),...,1,0,1,0,0,0,1640391,-122.448085,37.804952,0.001507
548476,PD36969155,2022-01-01 08:45:00,V5204A,REG TABS,1551 BAY STREET,CA,8MTL483,121.0,02/01/2023 12:00:00 AM,POINT (-122.43249 37.80311),...,1,0,1,0,0,0,1612113,-122.432504,37.803162,0.000057


In [36]:
df_joined['segment_id'] = df_joined['Corridor'] + ' from ' + df_joined['Limits']
df_joined.head()

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,...,Week2,Week3,Week4,Week5,Holidays,BlockSweepID,segment_midpoint_x,segment_midpoint_y,dist,segment_id
171693,PD36185715,2022-01-01 02:45:00,PK6.01G,NOPRK 10P6,1000 GREAT HIGHWAY,CA,7GXP111,23.0,03/15/2022 12:00:00 AM,POINT (-122.51054 37.76982),...,1,1,1,1,1,1601828,-122.510437,37.767239,0.000233,Great Hwy from John F Kennedy Dr - Lincoln Way
1119962,PD36997575,2022-01-01 02:45:00,V5204A,REG TABS,2251 GREENWICH,CA,8XOD966,341.0,02/06/2023 12:00:00 AM,POINT (-122.43672 37.79858),...,0,1,0,0,0,1638225,-122.436596,37.798382,0.000214,Pixley St from Fillmore St - Steiner St
394815,946687140,2022-01-01 07:00:00,V22500E,DRIVEWAY,665 EDDY ST,CA,7SCR930,110.0,02/05/2023 12:00:00 AM,POINT (-122.41862 37.78289),...,1,1,1,1,0,1630919,-122.418355,37.783206,0.000282,Eddy St from Larkin St - Polk St
273898,PD36969144,2022-01-01 08:16:00,V5204A,REG TABS,100 YACHT ROAD,CA,7NVL746,121.0,11/20/2022 12:00:00 AM,POINT (-122.44857 37.80643),...,0,1,0,0,0,1640391,-122.448085,37.804952,0.001507,Marina Blvd from Marina Blvd - Lyon St
548476,PD36969155,2022-01-01 08:45:00,V5204A,REG TABS,1551 BAY STREET,CA,8MTL483,121.0,02/01/2023 12:00:00 AM,POINT (-122.43249 37.80311),...,0,1,0,0,0,1612113,-122.432504,37.803162,0.000057,Bay St from Laguna St - Buchanan St


In [38]:
# let's just starting modeling on only Tuesdays
# Now stratify by days of the week.
df_joined['Day of Week'] = df_joined['Citation Issued DateTime'].dt.day_name()

In [40]:
df_joined.head()

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,...,Week3,Week4,Week5,Holidays,BlockSweepID,segment_midpoint_x,segment_midpoint_y,dist,segment_id,Day of Week
171693,PD36185715,2022-01-01 02:45:00,PK6.01G,NOPRK 10P6,1000 GREAT HIGHWAY,CA,7GXP111,23.0,03/15/2022 12:00:00 AM,POINT (-122.51054 37.76982),...,1,1,1,1,1601828,-122.510437,37.767239,0.000233,Great Hwy from John F Kennedy Dr - Lincoln Way,Saturday
1119962,PD36997575,2022-01-01 02:45:00,V5204A,REG TABS,2251 GREENWICH,CA,8XOD966,341.0,02/06/2023 12:00:00 AM,POINT (-122.43672 37.79858),...,1,0,0,0,1638225,-122.436596,37.798382,0.000214,Pixley St from Fillmore St - Steiner St,Saturday
394815,946687140,2022-01-01 07:00:00,V22500E,DRIVEWAY,665 EDDY ST,CA,7SCR930,110.0,02/05/2023 12:00:00 AM,POINT (-122.41862 37.78289),...,1,1,1,0,1630919,-122.418355,37.783206,0.000282,Eddy St from Larkin St - Polk St,Saturday
273898,PD36969144,2022-01-01 08:16:00,V5204A,REG TABS,100 YACHT ROAD,CA,7NVL746,121.0,11/20/2022 12:00:00 AM,POINT (-122.44857 37.80643),...,1,0,0,0,1640391,-122.448085,37.804952,0.001507,Marina Blvd from Marina Blvd - Lyon St,Saturday
548476,PD36969155,2022-01-01 08:45:00,V5204A,REG TABS,1551 BAY STREET,CA,8MTL483,121.0,02/01/2023 12:00:00 AM,POINT (-122.43249 37.80311),...,1,0,0,0,1612113,-122.432504,37.803162,0.000057,Bay St from Laguna St - Buchanan St,Saturday


In [51]:
df_joined = df_joined[df_joined['Day of Week'] == "Tuesday"]
df_joined.head()

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,...,Week3,Week4,Week5,Holidays,BlockSweepID,segment_midpoint_x,segment_midpoint_y,dist,segment_id,Day of Week
287268,PD37009420,2022-01-04 00:00:00,V5204A,REG TABS,101 GROVE STREET,CA,99833C1,231.0,09/11/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.777870,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
559229,946786750,2022-01-04 00:04:00,TRC7.2.22,STR CLEAN,465 CLEMENTINA ST,CA,6EWY058,85.0,04/10/2022 12:00:00 AM,POINT (-122.40532 37.77953),...,1,1,1,0,1603152,-122.405042,37.779821,0.000053,Clementina St from 05th St - 06th St,Tuesday
154949,946974453,2022-01-04 00:05:00,TRC7.2.22,STR CLEAN,4 BLUXOME ST,CA,C049707,85.0,03/02/2022 12:00:00 AM,POINT (-122.39590 37.77771),...,1,1,1,0,1612906,-122.396137,37.778013,0.000094,04th St from Brannan St - Bluxome St,Tuesday
503338,946977544,2022-01-04 00:05:00,TRC7.2.22,STR CLEAN,80 STEVENSON ST,CA,6SZZ016,85.0,01/13/2022 12:00:00 AM,POINT (-122.40043 37.78922),...,1,1,1,0,1624619,-122.400081,37.789393,0.000077,Stevenson St from Ecker St - 02nd St,Tuesday
804604,946786761,2022-01-04 00:06:00,TRC7.2.22,STR CLEAN,410 CLEMENTINA ST,CA,7WOW232,85.0,02/22/2022 12:00:00 AM,POINT (-122.40422 37.78055),...,1,1,1,0,1603152,-122.405042,37.779821,0.000071,Clementina St from 05th St - 06th St,Tuesday


In [92]:
df_joined['segment_id'] = df_joined['segment_id'].astype(str)

# Replace double spaces with single spaces in column 'A'
df_joined['segment_id'] = df_joined['segment_id'].str.replace('  ', ' ')

df_joined.head()

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,...,Week3,Week4,Week5,Holidays,BlockSweepID,segment_midpoint_x,segment_midpoint_y,dist,segment_id,Day of Week
287268,PD37009420,2022-01-04 00:00:00,V5204A,REG TABS,101 GROVE STREET,CA,99833C1,231.0,09/11/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.777870,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
559229,946786750,2022-01-04 00:04:00,TRC7.2.22,STR CLEAN,465 CLEMENTINA ST,CA,6EWY058,85.0,04/10/2022 12:00:00 AM,POINT (-122.40532 37.77953),...,1,1,1,0,1603152,-122.405042,37.779821,0.000053,Clementina St from 05th St - 06th St,Tuesday
154949,946974453,2022-01-04 00:05:00,TRC7.2.22,STR CLEAN,4 BLUXOME ST,CA,C049707,85.0,03/02/2022 12:00:00 AM,POINT (-122.39590 37.77771),...,1,1,1,0,1612906,-122.396137,37.778013,0.000094,04th St from Brannan St - Bluxome St,Tuesday
503338,946977544,2022-01-04 00:05:00,TRC7.2.22,STR CLEAN,80 STEVENSON ST,CA,6SZZ016,85.0,01/13/2022 12:00:00 AM,POINT (-122.40043 37.78922),...,1,1,1,0,1624619,-122.400081,37.789393,0.000077,Stevenson St from Ecker St - 02nd St,Tuesday
804604,946786761,2022-01-04 00:06:00,TRC7.2.22,STR CLEAN,410 CLEMENTINA ST,CA,7WOW232,85.0,02/22/2022 12:00:00 AM,POINT (-122.40422 37.78055),...,1,1,1,0,1603152,-122.405042,37.779821,0.000071,Clementina St from 05th St - 06th St,Tuesday


In [99]:
# hayes st from webster to buchanan
temp_df = df_joined[df_joined['segment_id'] == "Lech Walesa St from Polk St - Van Ness Ave"]

In [101]:
temp_df.head(50)

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,...,Week3,Week4,Week5,Holidays,BlockSweepID,segment_midpoint_x,segment_midpoint_y,dist,segment_id,Day of Week
287268,PD37009420,2022-01-04 00:00:00,V5204A,REG TABS,101 GROVE STREET,CA,99833C1,231.0,09/11/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.77787,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
287253,PD37009431,2022-02-01 00:00:00,TRC7.2.41,PK PHB OTD,101 GROVE STREET,CA,5LZZ399,108.0,05/04/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.77787,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
287252,1005782606,2022-02-01 10:52:00,TRC7.2.42,RESTRICTED,101 GROVE STREET,CA,8ZIW379,108.0,03/09/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.77787,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
287283,1005782621,2022-02-01 14:21:00,TRC7.2.42,RESTRICTED,101 GROVE STREET,CA,8PNG295,108.0,02/14/2023 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.77787,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
287254,PD37009442,2022-02-08 00:00:00,TRC7.2.41,PK PHB OTD,101 GROVE STREET,CA,5HUP133,108.0,03/22/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.77787,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
287260,1005782691,2022-02-08 12:32:00,TRC7.2.42,RESTRICTED,101 GROVE STREET,CA,5PEY289,108.0,03/28/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.77787,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
287258,PD37009501,2022-02-15 00:00:00,TRC7.2.40,PRK PROHIB,101 GROVE STREET,CA,7NRY430,108.0,03/21/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.77787,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
287265,PD37009781,2022-03-08 00:00:00,TRC7.2.40,PRK PROHIB,101 GROVE STREET,CA,1465070,108.0,03/29/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.77787,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
287264,PD37009766,2022-03-08 00:00:00,TRC7.2.40,PRK PROHIB,101 GROVE STREET,CA,142Z112,108.0,05/18/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.77787,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
287261,1005786040,2022-03-08 09:41:00,TRC7.2.42,RESTRICTED,101 GROVE STREET,CA,7YKC264,108.0,04/06/2022 12:00:00 AM,POINT (-122.41871 37.77812),...,1,1,1,0,1638976,-122.418925,37.77787,0.000222,Lech Walesa St from Polk St - Van Ness Ave,Tuesday
